In [ ]:
# !pip install transformers

In [1]:
from collections import defaultdict

import sys, gc, time
import numpy as np
# import pandas as pd
import torch
cuda = torch.device('cuda')
# import transformers as ppb
from transformers import RobertaForMaskedLM, RobertaTokenizer, pipeline
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
model = RobertaForMaskedLM.from_pretrained("roberta-large")
model.eval()
model.to('cuda')
# roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
# roberta.eval()
# from fairseq.data.data_utils import collate_tokens
# from fairseq import utils

from anytree import Node, RenderTree
import xml.etree.ElementTree as ET

import nltk
nltk.download('words')
nltk.download('punkt')
nltk.download('semcor') #semcor 3.0
nltk.download('wordnet') #wordnet 3.0?
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import semcor
from nltk.corpus import LazyCorpusLoader, SemcorCorpusReader
from nltk.corpus import words

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# import seaborn as sns
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
# from sklearn.cluster import DBSCAN
# from sklearn import metrics
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import train_test_split

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Mate\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mate\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package semcor to
[nltk_data]     C:\Users\Mate\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mate\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
VERBOSE = False
vprint = print if VERBOSE else lambda *a, **k: None

In [3]:
VOCAB_SIZE = 50265
id2word = np.array([tokenizer.decode(i) for i in range(VOCAB_SIZE)])
id2word_mod = np.array([tokenizer.decode(i).strip().lower() for i in range(VOCAB_SIZE)])
id2wordmask = np.where(np.isin(id2word_mod, words.words()), 1, 0)
# print(tokenizer.encode(id2word[24]))
word2id = {k: v for v, k in enumerate(id2word)}

In [4]:
# Fill mask, handles batches
# masked_input must be a list
def fill_mask_hint(masked_input, topk: int = 20):
    MASK_POS = 1
    PAD_TOK_ENC = 1
    
    with torch.no_grad():
        
        encoded = [tokenizer.encode(sentence, add_special_tokens=True) for sentence in masked_input]
        vprint("encoded\n", encoded)

        max_len = len(max(encoded, key=len))
        padded = np.array([i + [PAD_TOK_ENC]*(max_len-len(i)) for i in encoded])
        vprint("padded\n", padded)
        attention_mask = np.where(padded != PAD_TOK_ENC, np.ones(padded.shape), np.zeros(padded.shape))
        vprint("attn\n", attention_mask)

        input_ids = torch.LongTensor(padded).to('cuda')
        attention_mask = torch.tensor(attention_mask).to('cuda')

        vprint("padded\n", input_ids)
        vprint("attention_mask\n", attention_mask)
        
#         vprint("decodedtemp\n",[tokenizer.decode(input_id) for input_id in input_ids])

        #assumes one mask per sentence
#         mask_token_index = torch.nonzero(input_ids == tokenizer.mask_token_id, as_tuple=False)[:,1:].squeeze()
    
        token_logits = model(input_ids, attention_mask=attention_mask)[0]
        probs = token_logits.softmax(dim=-1)

        vprint("probs\n", probs)
        vprint("probs.size()\n", probs.size())

        mask_token_probs = probs.cpu().numpy()[:, MASK_POS, :] #fishy
        vprint("mask_token_probs\n", mask_token_probs)
        
        mask_token_probs = torch.tensor(mask_token_probs)
        mask_token_probs = mask_token_probs * torch.tensor(id2wordmask)
        mask_token_probs = mask_token_probs.fill_diagonal_(0)
        s = torch.sum(mask_token_probs, dim=1).unsqueeze(1)
        vprint("s\n",s)
        mask_token_probs /= s
        vprint("probs rowsum\n", torch.sum(mask_token_probs, dim=1))
        vprint("probs\n", mask_token_probs)
    
        # Return unsorted probs for all vocab words
        if topk==-1:
            return mask_token_probs, None
        
        top_probs, top_ids = torch.topk(torch.tensor(mask_token_probs), topk, dim=1)

#     top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
#     decoded = np.zeros_like(top_5_tokens)
#     for i, row in enumerate(top_5_tokens):
#         for j, token in enumerate(row):
#             decoded[i,j] = tokenizer.decode([token])
    
    return top_probs.numpy(), top_ids.numpy()

In [5]:
# Getting mask-fill probabilities without hint, and mask-fill probabilities after simple hint from all words in vocab

prior = np.zeros((1, VOCAB_SIZE), dtype=float)
init_probs, _ = fill_mask_hint([f'{tokenizer.mask_token} is a type of'], topk=-1)
prior[0] = init_probs
vprint("prior\n", prior)

start = time.perf_counter()

probs = np.zeros((VOCAB_SIZE, 20), dtype=float)
ids = np.zeros((VOCAB_SIZE, 20), dtype=int)
batch = []
i_beg = 0
for i, word in enumerate(id2word):
    if i%10000 == 0:
        print("iter:",i)
    
    sentence = f'{tokenizer.mask_token} is a type of{word}.'
    vprint(sentence)
    if i == 3195:
        print("sentence\n",sentence)
    batch.append(sentence)
    
    if i%32 == 0 or i==VOCAB_SIZE-1:
        vprint("Running through Roberta...")
        top_probs, top_ids = fill_mask_hint(batch)
        probs[i_beg:i+1] = top_probs.squeeze()
        ids[i_beg:i+1] = top_ids.squeeze()
        batch = []
        i_beg = i+1
        
end = time.perf_counter()

print("~ Time elapsed:", end-start)

print(probs)

# print("===============================================================")
# fills = fill_mask_hint(roberta, 'The sky is <mask>. <mask> is a color.', topk=10)
# print(fills)

iter: 0


c:\users\mate\documents\upenn\cis 700\finalproj\venv\lib\site-packages\ipykernel_launcher.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


sentence
 <mask> is a type of color.
iter: 10000
iter: 20000
iter: 30000
iter: 40000
iter: 50000
~ Time elapsed: 200.5756683999998
[[0.36681184 0.16774341 0.12888986 ... 0.00327605 0.00262635 0.0025966 ]
 [0.38128373 0.16491276 0.1088893  ... 0.00401988 0.00336012 0.00266258]
 [0.24865817 0.23445959 0.0982333  ... 0.00468203 0.00398821 0.0026309 ]
 ...
 [0.72297603 0.06327906 0.04267603 ... 0.00135634 0.00125626 0.00120673]
 [0.71807492 0.0638559  0.04194616 ... 0.00132    0.00126251 0.00122196]
 [0.42500779 0.09307698 0.05942865 ... 0.00334445 0.00324488 0.00315449]]


In [6]:
# Print prior and probs
vprint("Prior\n", prior)
vprint("Probs\n", probs)
vprint()

# Take difference with prior
vprint("Diffing...")
diff = probs - prior[0][ids]
vprint("diff\n",diff)
vprint()

# Argsort descending on columns
sorted_diff = np.sort(diff)[:,::-1]
sorted_diff_i = np.argsort(diff)[:,::-1]
sorted_diff_ids = np.take_along_axis(ids.astype('int'), sorted_diff_i, axis=-1)
sorted_diff_words = id2word[sorted_diff_ids]
vprint("sorted_diff_words\n",sorted_diff_words)
vprint()

vprint("sorted_diff_words vs id2word[ids] for 2")
vprint("sorted_diff_words\n",sorted_diff_words[2])
vprint("id2word[ids]\n",id2word[ids[2]])
vprint()

# Take quotient with prior
vprint("Dividing...")
div = probs/prior[0][ids]
vprint("div\n",div)
vprint()

# Argsort descending on columns
sorted_div = np.sort(div)[:,::-1]
sorted_div_i = np.argsort(div)[:,::-1]
sorted_div_ids = np.take_along_axis(ids.astype('int'), sorted_div_i, axis=-1)
sorted_div_words = id2word[sorted_div_ids]
vprint("sorted_div_words\n",sorted_div_words)
vprint()

vprint("sorted_div_words vs id2word[ids] for 2")
vprint("sorted_div_words\n",sorted_div_words[2])
vprint("id2word[ids]\n",id2word[ids[2]])
vprint()

In [9]:
###########################
# Optional cell for testing!
###########################

# Test with word " color"
print(word2id[" color"])

print("Compare abs diffs")
print("sorted_diff[3195]\n",sorted_diff[3195])
print("sorted_diff[2]\n",sorted_diff[2])
print("sorted_diff[5412]\n",sorted_diff[5412])

print("Compare abs divs")
print("sorted_div[3195]\n",sorted_div[3195])
print("sorted_div[2]\n",sorted_div[2])
print("sorted_div[5412]\n",sorted_div[5412])

# id2word[tokenizer.encode("color")]
colorid = 3195
#Sorted based on diff with prior
print(sorted_diff_words[colorid])
#Sorted based on topk probs of mask
print(id2word[ids[colorid]])

with torch.no_grad():
    token_logits = model(torch.tensor(tokenizer.encode(f"{tokenizer.mask_token} is a type of color.{tokenizer.pad_token}")).unsqueeze(0).to("cuda"), attention_mask=torch.tensor([1,1,1,1,1,1,1,1,1,0]).unsqueeze(0).to("cuda"))[0][0][1]
    #Sorted based on topk probs of mask
    print(id2word[torch.topk(token_logits.softmax(dim=-1), 20).indices.tolist()])
    #Prior only
    print(id2word[torch.topk(torch.tensor(prior), 20).indices.tolist()])

3195
Compare abs diffs
sorted_diff[3195]
 [ 0.10996049  0.10266602  0.07808552  0.07705928  0.06087634  0.051904
  0.03881612  0.03767307  0.0343217   0.01759202  0.01627531  0.01562993
  0.0111918   0.01114852  0.00389159  0.0038626  -0.0332558  -0.04757096
 -0.14782091 -0.16417575]
sorted_diff[2]
 [ 0.14819297  0.04553045  0.02501446  0.02453257  0.02024946  0.01781523
  0.01331211  0.0103311   0.0038702   0.00319078  0.00266973  0.00181848
  0.00179262  0.00067157 -0.00061546 -0.00369854 -0.00380379 -0.00396616
 -0.05007261 -0.06721348]
sorted_diff[5412]
 [ 1.28536269e-01  2.68400759e-02  1.93208782e-02  1.16665622e-02
  1.10092771e-02  1.09011538e-02  7.73921551e-03  6.55087299e-03
  5.24582207e-03  5.08044666e-03  4.71859123e-03  4.61943622e-03
  4.39793320e-03  2.97113135e-03  7.20875338e-04  1.13043934e-05
 -1.20615587e-05 -1.03308726e-03 -4.99665621e-02 -1.20786924e-01]
Compare abs divs
sorted_div[3195]
 [6.32436995e+02 4.95674999e+02 3.69001373e+02 3.40555185e+02
 2.49446508e+

c:\users\mate\documents\upenn\cis 700\finalproj\venv\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [32]:
##############################
# Optional cell for tree demo!
##############################

udo = Node("Udo")
marc = Node("Marc", parent=udo)
lian = Node("Lian", parent=marc)
dan = Node("Dan", parent=udo)
jet = Node("Jet", parent=dan)
jan = Node("Jan", parent=dan)
joe = Node("Joe", parent=dan)
# print(udo)
print(RenderTree(udo))
joe.parent = marc
marc.parent = dan
print(RenderTree(udo))
print(joe.name)

Node('/Udo')
├── Node('/Udo/Marc')
│   └── Node('/Udo/Marc/Lian')
└── Node('/Udo/Dan')
    ├── Node('/Udo/Dan/Jet')
    ├── Node('/Udo/Dan/Jan')
    └── Node('/Udo/Dan/Joe')
Node('/Udo')
└── Node('/Udo/Dan')
    ├── Node('/Udo/Dan/Jet')
    ├── Node('/Udo/Dan/Jan')
    └── Node('/Udo/Dan/Marc')
        ├── Node('/Udo/Dan/Marc/Lian')
        └── Node('/Udo/Dan/Marc/Joe')
Joe


In [27]:
# Build skeleton tree
root = Node("<r>")
nodes = []
print("adding initial nodes")
for i, word in enumerate(id2word):
    if i%10000 == 0:
        print("iter:", i)
    # i is not in readable vocab
    if id2wordmask[i] == 0:
        node = Node(word, parent=None)
    else:
        node = Node(word, parent=root)
    nodes.append(node)
# print(nodes)

adding initial nodes
iter: 0
iter: 10000
iter: 20000
iter: 30000
iter: 40000
iter: 50000


In [28]:
# Filter based on criterion above
DIV_THRESH = 1000
DIFF_THRESH = .03
filtered = np.where(sorted_div > DIV_THRESH, sorted_div_ids, -1)
assert filtered.shape[0] == sorted_div.shape[0] == id2wordmask.size
# print(filtered)

print(filtered.shape)

(50265, 20)


In [29]:
# Parent-child boolean matrix, shape(N,N)
Mfiltered = np.zeros((filtered.shape[0], filtered.shape[0]), dtype=bool)

# print("adding new parents")
for i_parent, filtered_row in enumerate(filtered):
    if i%10000 == 0:
        print("iter:", i_parent)
#     if id2wordmask[i] == 0:
#         nodes[i].parent = None
#         continue
    # none in ith row passes threshold
    if np.all(filtered_row == -1):
        continue
    for i_child in filtered_row[filtered_row != -1]:
        # Case i_child is on diagonal, skip
        if i_child == i_parent:
            continue
        # Case i_child is above diagonal
        elif i_child > i_parent:
            Mfiltered[i_parent,i_child] = True
        # Case i_child is below diagonal
        else:
            # Check transpose element. If True -> cohyponym
            if Mfiltered[i_child,i_parent]:
                Mfiltered[i_child,i_parent] = False
            else:
                Mfiltered[i_parent,i_child] = True
#             print(nodes[_id], nodes[i])
#             nodes[_id].parent = nodes[i]

In [30]:
nz = Mfiltered.nonzero()
print(len(nz))
print(len(nz[0]))
for i,(r,c) in enumerate(zip(nz[0],nz[1])):
    if i%10000 == 0:
        print("iter:", i)
    vprint(nodes[c].name.strip().lower(), nodes[r].name.strip().lower())
    if nodes[c].name.strip().lower() != nodes[r].name.strip().lower():
        nodes[c].parent = nodes[r]
        print(nodes[c], nodes[r])

2
2387
iter: 0
Node('/<r>/ if/Else') Node('/<r>/ if')
Node('/<r>/ want/Need') Node('/<r>/ want')
Node('/<r>/ State/Republic') Node('/<r>/ State')
Node('/<r>/ city/Town') Node('/<r>/ city')
Node('/<r>/ night/Tonight') Node('/<r>/ night')
Node('/<r>/ City/Town') Node('/<r>/ City')
Node('/<r>/ County/City') Node('/<r>/ County')
Node('/<r>/ County/Town') Node('/<r>/ County')
Node('/<r>/ County/District') Node('/<r>/ County')
Node('/<r>/ money/Cash') Node('/<r>/ money')
Node('/<r>/ media/Music') Node('/<r>/ media')
Node('/<r>/ media/Film') Node('/<r>/ media')
Node('/<r>/ media/Gaming') Node('/<r>/ media')
Node('/<r>/ House/Senate') Node('/<r>/ House')
Node('/<r>/ give/Giving') Node('/<r>/ give')
Node('/<r>/ major/Minor') Node('/<r>/ major')
Node('/<r>/ political/Politics') Node('/<r>/ political')
Node('/<r>/ West/East') Node('/<r>/ West')
Node('/<r>/ University/College') Node('/<r>/ University')
Node('/ Facebook/Twitter') Node('/ Facebook')
Node('/<r>/ trade/Deal') Node('/<r>/ trade')
Node(

In [25]:
# Precision: tree -> WN
TP1 = 0
FP = 0
error_sample = []
# for i,(r,c) in enumerate(zip(nz[0],nz[1])):
for i,n in enumerate(nodes):
    if n.parent == None or n.parent.name == "<r>":
        continue
    parent = n.parent.name.strip().lower()
    child = n.name.strip().lower()
    
    if len(wn.synsets(parent)) > 0 and len(wn.synsets(child)) > 0:
        wnparent = wn.synsets(parent)[0]
        wnchild = wn.synsets(child)[0]
    else:
        continue
    vprint(wnparent, wnchild)
    if len(wnchild.hypernyms()) > 0 and wnparent in wnchild.hypernyms():
        TP1 += 1
    else:
        if np.random.uniform() < .1:
            error_sample.append((child, parent))
        FP += 1
        
print(TP1)
print(FP)
print(np.array(error_sample))

10
290
[['day' 'time']
 ['right' 'left']
 ['rain' 'precipitation']
 ['set' 'sets']
 ['tonight' 'night']
 ['return' 'returning']
 ['hay' 'fodder']
 ['magic' 'alchemy']
 ['sad' 'miserable']
 ['brook' 'swamp']
 ['iron' 'creatine']
 ['freedom' 'liberty']
 ['giving' 'give']
 ['rum' 'soda']
 ['friend' 'friendship']
 ['vim' 'vi']
 ['round' 'rounded']
 ['truth' 'truths']
 ['ocean' 'oceans']
 ['return' 'returning']
 ['catholic' 'orthodox']
 ['crystal' 'mdma']
 ['talking' 'communicating']
 ['movie' 'movies']
 ['region' 'province']
 ['pain' 'agony']
 ['luck' 'unlucky']
 ['beast' 'wolverine']
 ['bone' 'bones']
 ['ruby' 'python']
 ['home' 'house']
 ['material' 'matter']
 ['myth' 'myths']
 ['optional' 'option']
 ['python' 'java']
 ['instance' 'instances']]


In [ ]:
# Recall: WN -> tree

TP2 = 0
FN = 0
error_sample = []
for ss in wn.all_synsets():
    if("_" not in ss.name()):
        child = ss.lemmas()[0].name()
        if child not in word2id:
            continue
        child_id = word2id[child]
        if nodes[child_id].parent is None or nodes[child_id].parent.name == "<r>":
            continue
        hypernyms = [h.lemmas()[0].name() for h in ss.hypernyms() if "_" not in h.name()]
        hypernyms = [word2id[h] for h in hypernyms if h in word2id]
        if len(hypernyms) == 0:
            continue
        for parent_id in hypernyms:
            parent = id2word[parent_id]
            if nodes[child_id].parent.name.strip().lower() == parent.strip().lower():
                TP2 += 1
            else:
                if np.random.uniform() < .05:
                    error_sample.append((child, parent))
                vprint(word, id2word[parent_id])
                vprint(child_id, parent_id)
                FN +=1
            
#                            all("_" not in s.name() for s in ss.hypernyms())):
print(TP2)
print(FN)
print(np.array(error_sample))

# print(nodes[word2id["default"]].parent)
# print(nodes[word2id["header"]].parent)
# print(nodes[word2id["choice"]].parent)
# print(nodes[word2id["goal"]].parent)

In [ ]:
print(RenderTree(root))

In [271]:
# Average sum of probabilities for each row for top-k
np.average(np.sum(probs, axis=1))

0.776035287558173

!!!!!!!!!





This is the end of the current code. Below is code used for abandonded ideas or experimentation.





!!!!!!!!!

In [ ]:
#Download pretrained model and weights
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
#Gather pure and sense dataset
N = 3000
sents_pure = np.array(semcor.sents()[0:N])
sents_senses = np.array(semcor.tagged_sents(tag="sem")[0:N])
print(sents_pure[0])
print(sents_senses[0])

# print(sents_senses[2][0] in sents_senses[2].leaves())
# print(sents_senses[2])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', 'Atlanta', "'s", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
[['The'], Tree('group.n.01', [Tree('NE', ['Fulton', 'County', 'Grand', 'Jury'])]), Tree('say.v.01', ['said']), Tree('friday.n.01', ['Friday']), ['an'], Tree('investigation.n.01', ['investigation']), ['of'], Tree('atlanta.n.01', ['Atlanta']), ["'s"], Tree('recent.s.02', ['recent']), Tree('primary_election.n.01', ['primary', 'election']), Tree('produce.v.04', ['produced']), ['``'], ['no'], Tree('evidence.n.01', ['evidence']), ["''"], ['that'], ['any'], Tree('irregularity.n.01', ['irregularities']), Tree('take_place.v.01', ['took', 'place']), ['.']]


In [ ]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

batch_1 = df[:5000]
print(batch_1.iloc[:10,0])
print(batch_1[1].value_counts())

#Tokenize
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized

#Padding for batch
max_len = tokenized.str.len().max()
print(max_len)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
padded

#Mask the padding
attention_mask = np.where(padded != 0, 1, 0)
attention_mask

#Run through model, save last hidden states
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

#Prep for regression
print(last_hidden_states[0].shape)
features = last_hidden_states[0][:,0,:].numpy()
labels = batch_1[1]

#Split, fit, and score
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)

In [ ]:
#Tokenize
tokenized = [tokenizer.encode(x, add_special_tokens=False) for x in sents_pure]
print(len(tokenized[0]), len(sents_pure[0]))
print(tokenized[0])

#Padding for batch
max_len = len(max(tokenized, key=len))
print(max_len)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized])
print(padded[0])

#Mask the padding
attention_mask = np.where(padded != 0, 1, 0)
print(attention_mask[0])

26 26
[100, 100, 100, 100, 100, 2056, 100, 2019, 4812, 1997, 100, 100, 3522, 3078, 2602, 2550, 100, 2053, 3350, 100, 2008, 2151, 28868, 2165, 2173, 1012]
138
[  100   100   100   100   100  2056   100  2019  4812  1997   100   100
  3522  3078  2602  2550   100  2053  3350   100  2008  2151 28868  2165
  2173  1012     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]
[

In [ ]:
#Run through model, save last hidden states
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
for sent in sents_senses:
    for word in sent:
        if (isinstance(word, nltk.tree.Tree)
        and word[0] in word.leaves()
        and len(word) == 1):
            print(word.label())
            print(word[0])
            print("---")

In [ ]:
#Build data structs
word2senses = defaultdict(set)
senses2embeddings = defaultdict(list)
sents_filteredsenses = []
for i, (sent, sent_hidden) in enumerate(zip(sents_senses, last_hidden_states[0])):
    newsent = []
    for word, word_hidden in zip(sent, sent_hidden):
        if (isinstance(word, nltk.tree.Tree)
        and word[0] in word.leaves()
        and len(word) == 1):
            newsent.append(word.label())
            word2senses[word[0]].add(word.label())
            print(word2senses[word[0]])
            senses2embeddings[word.label()].append(word_hidden)
        else:
            newsent.append(None)
    sents_filteredsenses.append(newsent)

In [ ]:
print(sents_filteredsenses[0])
print(word2senses)
print()
# print(len(senses2embeddings["say.v.01"]))
# print(len(senses2embeddings["say.v.02"]))

for k in word2senses:
    if len(word2senses[k]) > 1:
        print(k, ":", word2senses[k])

print(word2senses["like"])
print(word2senses["thanks"])
print()

#tSNE plots of selected words
for word in ["like", "thanks", "man", "let"]:
    X = []
    labels = []
    i=0
    for sense in word2senses[word]:
        # print(senses2embeddings[sense][0].numpy())
        embeddings = senses2embeddings[sense]
        X += embeddings
        labels += [i]*len(embeddings)
        i+=1
        # X += [embedding.numpy() for embedding in senses2embeddings[sense]]
    X = torch.stack(X).numpy()
    labels = np.array(labels)[:,np.newaxis]
    print(X.shape)

    # pca_50 = PCA(n_components=50)
    # pca_result_50 = pca_50.fit_transform(X)
    # print('Cumulative explained variation for 50 principal components: {}'.format(np.sum(pca_50.explained_variance_ratio_)))
    tsne = TSNE(n_components=2, verbose=0, perplexity=40, n_iter=400)
    tsne_pca_results = tsne.fit_transform(X)

    tsne_pca_results = np.concatenate((tsne_pca_results, labels), axis=1)
    colors = np.array(['tab:blue', 'tab:orange', 'tab:green'])
    fig, ax = plt.subplots()
    scatter = ax.scatter(tsne_pca_results[:,0],
                        tsne_pca_results[:,1],
                        c=colors[tsne_pca_results[:,2].astype(int)])

    ax.legend()
    ax.grid(True)
    plt.title("Gold clustering of " + word)

    plt.show()

# X = StandardScaler().fit_transform(X)
# db = DBSCAN(eps=0.3, min_samples=10).fit(X)
# core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
# core_samples_mask[db.core_sample_indices_] = True
# labels = db.labels_

# print(labels)

# # Number of clusters in labels, ignoring noise if present.
# n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
# n_noise_ = list(labels).count(-1)

# print('Estimated number of clusters: %d' % n_clusters_)
# print('Estimated number of noise points: %d' % n_noise_)
# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
# print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
# print("Adjusted Rand Index: %0.3f"
#       % metrics.adjusted_rand_score(labels_true, labels))
# print("Adjusted Mutual Information: %0.3f"
#       % metrics.adjusted_mutual_info_score(labels_true, labels))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, labels))

![say - WordNet results](https://drive.google.com/uc?id=1bWqulSz86LgD170F9YJWQn5zO52MGHUa)
![produce - WordNet results](https://drive.google.com/uc?id=12RE9iwemg97if7UuhKgLuUv2yEqOzAv0)




In [ ]:
####################################################
# Unused Code
####################################################
    
#####################
# Pytorch Hub Roberta
#####################

#     masked_token = '<mask>'
#     # assert masked_token in masked_input and masked_input.count(masked_token) == 1, \
#     #     "Please add one {0} token for the input, eg: 'He is a {0} guy'".format(masked_token)

#     text_spans = masked_input.split(masked_token)
#     text_spans_bpe = (' {0} '.format(masked_token)).join(
#         [roberta.bpe.encode(text_span.rstrip()) for text_span in text_spans]
#     ).strip()
#     tokens = roberta.task.source_dictionary.encode_line(
#         '<s> ' + text_spans_bpe + ' </s>',
#         append_eos=False,
#         add_if_not_exist=False,
#     )
    
#     vprint(text_spans)
#     vprint(text_spans_bpe)
#     vprint(tokens)
#     vprint(roberta.task.mask_idx)
#     vprint("===")

#     masked_index = (tokens == roberta.task.mask_idx).nonzero()
#     if tokens.dim() == 1:
#         tokens = tokens.unsqueeze(0)
        
#     vprint(masked_index)
#     vprint(tokens)
#     vprint("===mask+tokens")

#     with utils.eval(roberta.model):
#         features, extra = roberta.model(
#             tokens.long().to(device=roberta.device),
#             features_only=False,
#             return_all_hiddens=False,
#         )
#     logits = features[0, masked_index, :].squeeze()
#     prob = logits.softmax(dim=-1)

#     vprint(features)
#     vprint(logits)
#     vprint(prob)
#     vprint("===probs")

#     if topk == -1:
#         topk = prob.shape[-1]
#         vprint(prob.shape[-1])
#     values, index = prob.topk(k=topk)#, dim=0)
#     vprint(values)
#     topk_predicted_token_bpe = roberta.task.source_dictionary.string(index)
#     topk_predicted_token_bpe_split = np.array([t.split(" ") for t in topk_predicted_token_bpe.split("\n")])
#     vprint(values)
#     vprint(index)
#     vprint(topk_predicted_token_bpe)
#     vprint(topk_predicted_token_bpe_split)
#     vprint("===VI")

#     topk_filled_outputs = []
#     #there should typically be 1 mask
#     for mask in topk_predicted_token_bpe_split:
#         for index, predicted_token_bpe in enumerate(mask):
#             predicted_token = roberta.bpe.decode(predicted_token_bpe)
#             # Quick hack to fix https://github.com/pytorch/fairseq/issues/1306  
#             if predicted_token_bpe.startswith('\u2581'):
#                 predicted_token = ' ' + predicted_token
#             # if " {0}".format(masked_token) in masked_input:
#                 # topk_filled_outputs.append((
#                 #     masked_input.replace(
#                 #         ' {0}'.format(masked_token), predicted_token
#                 #     ),
#                 #     values[index].item(),
#                 #     predicted_token,
#                 # ))
#             # else:
#                 # topk_filled_outputs.append((
#                 #     masked_input.replace(masked_token, predicted_token),
#                 #     values[index].item(),
#                 #     predicted_token,
#                 # ))
#             topk_filled_outputs.append(predicted_token)
#     return topk_filled_outputs

######################

# # Encode input
# tokens = roberta.encode('Hello world!')
# print(tokens)

# # Extract the last layer's features
# last_layer_features = roberta.extract_features(tokens)
# assert last_layer_features.size() == torch.Size([1, 5, 1024])
# print(last_layer_features)

# # Extract all layer's features (layer 0 is the embedding layer)
# all_layers = roberta.extract_features(tokens, return_all_hiddens=True)
# assert len(all_layers) == 25
# assert torch.all(all_layers[-1] == last_layer_features)

# # Define batch
# batch = ["<mask> is a type of"]
# for v in vocabulary:
#     batch.append("<mask> is a type of " + v)

# # Batch tokenize and pad
# batch = collate_tokens(
#     [roberta.encode(hint) for hint in batch], pad_idx=1
# )

# fill_mask_hint(roberta, batch, topk=10)

# # Logprob of predictions
# logprobs = roberta.predict('mnli', batch)
# print(logprobs.argmax(dim=1))

# # MLM predictions
# fills = roberta.fill_mask('The first Star wars movie came out in <mask>', topk=3)
# print(fills)

########################

# Open files

# f = open("enwiki-latest-abstract.xml", "r", encoding='utf-8')
# tree = ET.parse('enwiki-latest-abstract.xml')
# root = tree.getroot()
# del tree
# g = open("full.txt", "r", encoding='utf-8')
# g.close()
# f.close()

# i = 0
# for line in g.readlines():
#     print(line)
#     i+=1
#     if i%10 == 0:
#         raise KeyboardInterrupt

# Read Wikipedia abstracts

# abstract_dict = {}
# for doc in root.iter('doc'):
#     title = doc.find("title").text.lower()
#     abstract = doc.find("abstract").text
#     if abstract:
#         abstract = word_tokenize(abstract)
    
#     if "wikipedia: " in title:
#         title = title[len("wikipedia: "):]
        
#     abstract_dict[title] = abstract
    
# with open("enwiki-latest-abstract.xml", encoding='utf-8') as f:
# i = 0
# for line in f.readlines():
#     print(line)
#     i+=1
#     if i%100 == 0:
#         raise KeyboardInterrupt

# print(abstract_dict["anarchy"])

#########################

# Random tests of the encoder/decoder

# # tokenizer.decode(torch.tensor([0,1,2,3,4,5,6,7,8,9,10,50264,50263,50262,50261,50260,50259]))
# print(tokenizer.decode(torch.tensor(np.arange(16))))
# print(tokenizer.decode(torch.tensor(np.array([    0 ,50264  ,  16 ,   10 , 1907  ,   9 , 7586    , 2    , 1]))))
# print(tokenizer.decode(torch.tensor(np.array([    0, 50264   , 16  ,  10,  1907   ,  9  ,2156 ,    4     ,2   ,  1]))))
# print(tokenizer.decode(torch.tensor(np.array([    0 ,50264,    16 ,   10 , 1907,     9, 29942,     2  ,   1   ,  1]))))
# print(id2word[:10])
# # tokenizer.decode(tokenizer.encode("color"))

In [ ]:
#Try to find multiple hypernyms
for ss in wn.all_synsets():
    if(len(ss.hypernyms()) > 1 and
       "_" not in ss.name() and
       all("_" not in s.name() for s in ss.hypernyms())):
        print(ss)
        print(ss.hypernyms())
        print()

In [234]:
#Find max depth hypernyms
def get_depth(ss, depth):
    if len(ss.hypernyms()) == 0:
        return depth
    return get_depth(ss.hypernyms()[0], depth+1)

max_depth = 0
for i, ss in enumerate(wn.all_synsets()):
    if i%20000 == 0:
        print("Iter:",i)
    d = get_depth(ss, 1)
    if d > max_depth:
        max_depth = d

Iter: 0
Iter: 20000
Iter: 40000
Iter: 60000
Iter: 80000
Iter: 100000


In [235]:
max_depth

19

In [ ]:
wn.synsets('woman')[0].hyponyms()

[Synset('amazon.n.01'),
 Synset('b-girl.n.01'),
 Synset('bachelor_girl.n.01'),
 Synset('baggage.n.02'),
 Synset('ball-buster.n.01'),
 Synset('black_woman.n.01'),
 Synset('bluestocking.n.01'),
 Synset('bridesmaid.n.01'),
 Synset('broad.n.01'),
 Synset('cat.n.03'),
 Synset('cinderella.n.01'),
 Synset('coquette.n.01'),
 Synset('dame.n.02'),
 Synset('debutante.n.01'),
 Synset('divorcee.n.01'),
 Synset('dominatrix.n.01'),
 Synset('donna.n.01'),
 Synset('enchantress.n.01'),
 Synset('ex-wife.n.01'),
 Synset('eyeful.n.01'),
 Synset('geisha.n.01'),
 Synset('girl.n.01'),
 Synset('girl.n.05'),
 Synset('girlfriend.n.01'),
 Synset('girlfriend.n.02'),
 Synset('gold_digger.n.02'),
 Synset('gravida.n.02'),
 Synset('heroine.n.02'),
 Synset('inamorata.n.01'),
 Synset('jezebel.n.02'),
 Synset('jilt.n.01'),
 Synset('lady.n.01'),
 Synset('maenad.n.01'),
 Synset('maenad.n.02'),
 Synset('matriarch.n.01'),
 Synset('matriarch.n.02'),
 Synset('matron.n.03'),
 Synset('mestiza.n.01'),
 Synset('mistress.n.01'),
 S